# SageMaker/DeepAR Model Hyperparameter Tuning Demo
This notebook runs a hyperparameter tuning job to find the best parameters to train a DeepAR model. It then trains a model using those parameters, deploys the model and test it.

__You don't have to run this notebook__ and can directly use the the `model-creation-and-testing.ipynb` notebook, which uses results of a hyperparameter tuning job ran during our initial testing.

For the detailed explanation of the data transformation and usage of the DeepAR model, please refer to the `model-creation-and-testing.ipynb` notebook.

In [ ]:
from __future__ import print_function

%matplotlib inline

import os
import json
import random
import datetime

import boto3
import sagemaker
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from utils import DeepARPredictor, get_ssm_parameters
from sagemaker.session import Session
from sagemaker.feature_store.feature_group import FeatureGroup
from sagemaker.tuner import (
    HyperparameterTuner,
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
)
from sagemaker.serializers import IdentitySerializer
from sagemaker.deserializers import JSONDeserializer

## Initialisation
### Retrieve Parameters
We collect here information from ssm parameters about:
* the SageMake Feature Store storing our data
* model target
* model training hypertunning parameters

In [ ]:
def get_ssm_parameters(ssm_client, param_path):
    """Retrieves the SSM parameters from the specified path

    Args:
        ssm_client (botocore.client): The SSM client
        param_path (str): The path to the SSM parameters

    Returns:
        Dict[str, str]: The SSM parameters
    """
    parameters = {}
    try:
        response = ssm_client.get_parameters_by_path(
            Path=param_path, Recursive=False, WithDecryption=False
        )
        for param in response["Parameters"]:
            parameters[param["Name"].split("/")[-1]] = param["Value"]
        while next_token := response.get("NextToken"):
            response = ssm_client.get_parameters_by_path(
                Path=param_path,
                Recursive=False,
                WithDecryption=False,
                NextToken=next_token,
            )
            for param in response["Parameters"]:
                parameters[param["Name"].split("/")[-1]] = param["Value"]
    except Exception as e:
        print(f"An error occurred reading the SSM stack parameters: {e}")
    return parameters

In [ ]:
ssm_client = boto3.client("ssm")
# Read the Feature Group Name from SageMaker feature store
stack_parameters = get_ssm_parameters(ssm_client, "/rdi-mlops/stack-parameters")
# Read the SSM Parameters storing the parameters for the model training
target_parameters = get_ssm_parameters(ssm_client, "/rdi-mlops/sagemaker/model-build/target")

### SageMaker Initialisation

In [ ]:
# set random seeds for reproducibility
seed = 42
np.random.seed(seed)
random.seed(seed)

In [ ]:
# Set S3 Buckets variables
model_artifacts_bucket = f"{stack_parameters['project-prefix']}-sagemaker-experiment-{stack_parameters['bucket-suffix']}"
# Set prefix used for all data stored within the bucket
s3_model_prefix = "deepar-model"
train_validation_data_folder_prefix = "model-tuning"
s3_data_path = (
    f"s3://{model_artifacts_bucket}/data/{train_validation_data_folder_prefix}"
)
s3_output_path = f"s3://{model_artifacts_bucket}/{s3_model_prefix}/output"

In [ ]:
# Set session variables
sagemaker_session = sagemaker.Session(default_bucket=model_artifacts_bucket)
region = sagemaker_session.boto_region_name
account_id = sagemaker_session.account_id()
role = sagemaker.get_execution_role()
boto_session = boto3.Session(region_name=region)

In [ ]:
# Set boto3 S3 client
s3 = boto3.resource("s3")
# Set feature store session
sagemaker_client = boto_session.client(service_name="sagemaker", region_name=region)
featurestore_runtime = boto_session.client(
    service_name="sagemaker-featurestore-runtime", region_name=region
)

feature_store_session = Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime,
)

## Load Data From FeatureStore
The notebook uses Athena to load the data from the S3 bucket of the SageMaker Feature Store.

In [ ]:
transactions_feature_group = FeatureGroup(
    name=stack_parameters["sagemaker-feature-group-name"],
    sagemaker_session=feature_store_session,
)

In [ ]:
transactions_feature_group.describe()

Query the Data from FeatureStore using Athena

In [ ]:
transactions_data_query = transactions_feature_group.athena_query()
transactions_data_table = transactions_data_query.table_name

query_string = f'SELECT * FROM "{transactions_data_table}"'

# run Athena query. The output is loaded to a Pandas dataframe.
# dataset = pd.DataFrame()
transactions_data_query.run(
    query_string=query_string,
    output_location="s3://" + model_artifacts_bucket + "/query_results/",
)
transactions_data_query.wait()
df = transactions_data_query.as_dataframe()

Simple Dataset ETL:
- sort the values by time
- convert the time column from string to Pandas TImestamp

In [ ]:
df.sort_values(by="tx_minute", axis=0, ascending=True, inplace=True)
df["tx_minute"] = pd.to_datetime(df["tx_minute"])
df.set_index("tx_minute", drop=True, inplace=True)
df.head()

In [ ]:
print(f"Number of data points in the dataset: {len(df)}")

## Train, Test and Validation Splits
Prepare the train, test and validation datasets. Please refer to the `model-creation-and-testing.ipynb` notebook for details.

In [ ]:
start_dataset = df.index.min()
end_dataset = df.index.max()
freq = "1min"
dataset_period = end_dataset - start_dataset
num_validation_windows = int(target_parameters["num_validation_windows"])
target_col = target_parameters["target_col"]

In [ ]:
total_nb_data_points = len(df)
prediction_length = int(target_parameters["prediction_length"])
context_length = prediction_length
test_length = prediction_length
min_data_length = context_length + prediction_length * (num_validation_windows + 1)

In [ ]:
if total_nb_data_points < min_data_length:
    prediction_length = int(total_nb_data_points * 0.05)
    test_length = prediction_length
    context_length = total_nb_data_points - prediction_length * (
        num_validation_windows + 1
    )
validation_windows_length = num_validation_windows * prediction_length

### Data Formatting
The data have to be formated to the DeepAR format as described in [this documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/deepar.html#deepar-inputoutput).

In [ ]:
df_test = df[-test_length:]
df_train_validation = df[:-test_length]
df_train = df_train_validation[:-validation_windows_length]

training_data = [
    {
        "start": str(start_dataset),
        "target": list(df_train[target_col]),
    }
]
validation_data = [
    {
        "start": str(start_dataset),
        "target": list(
            df_train_validation.iloc[
                0 : -int((num_validation_windows - k) * prediction_length),
                df_train_validation.columns.get_loc(target_col),
            ]
        ),
    }
    for k in range(1, num_validation_windows)
]
validation_data.append(
    {
        "start": str(start_dataset),
        "target": list(df_train_validation[target_col]),
    }
)

We write the the train and test datasets JSON dictionaries file to S3

In [ ]:
def write_dicts_to_file(path, data):
    with open(path, "wb") as fp:
        for d in data:
            fp.write(json.dumps(d).encode("utf-8"))
            fp.write("\n".encode("utf-8"))

In [ ]:
if not os.path.isdir(train_validation_data_folder_prefix):
    os.mkdir(train_validation_data_folder_prefix)
write_dicts_to_file(f"{train_validation_data_folder_prefix}/train.json", training_data)
write_dicts_to_file(
    f"{train_validation_data_folder_prefix}/validation.json", validation_data
)

In [ ]:
def copy_to_s3(local_file, s3_path, override=False):
    assert s3_path.startswith("s3://")
    split = s3_path.split("/")
    bucket = split[2]
    path = "/".join(split[3:])
    buk = s3.Bucket(bucket)

    if len(list(buk.objects.filter(Prefix=path))) > 0:
        if not override:
            print(f"File {s3_path} already exists.\nSet override to upload anyway.\n")
            return
        else:
            print("Overwriting existing file")
    with open(local_file, "rb") as data:
        print(f"Uploading file to {s3_path}")
        buk.put_object(Key=path, Body=data)

In [ ]:
copy_to_s3(
    f"{train_validation_data_folder_prefix}/train.json",
    s3_data_path + "/train/train.json",
    override=True,
)
copy_to_s3(
    f"{train_validation_data_folder_prefix}/validation.json",
    s3_data_path + "/validation/validation.json",
    override=True,
)

## Train a Model
### Initialize the Model

In [ ]:
image_name = sagemaker.image_uris.retrieve("forecasting-deepar", region)
estimator = sagemaker.estimator.Estimator(
    image_uri=image_name,
    sagemaker_session=sagemaker_session,
    role=role,
    instance_count=1,
    instance_type="ml.c5.2xlarge",
    base_job_name=f"{stack_parameters['project-prefix']}-deepar",
    use_spot_instances=True,
    max_run=60 * 60 - 1,
    max_wait=60 * 60,
    output_path=s3_output_path,
)

data_channels = {"train": f"{s3_data_path}/train/", "test": f"{s3_data_path}/validation/"}

### Hyperparameters Tuning

In [ ]:
hyperparameters = {
    "time_freq": freq,
    "early_stopping_patience": "40",
    "context_length": str(context_length),
    "prediction_length": str(prediction_length),
}
estimator.set_hyperparameters(**hyperparameters)

hyperparameter_ranges = {
    "epochs": IntegerParameter(50, 400),
    "mini_batch_size": IntegerParameter(32, 128),
    "num_cells": IntegerParameter(30, 100),
    "likelihood": CategoricalParameter(["gaussian", "student-T"]),
    "learning_rate": ContinuousParameter(1e-5, 1e-2),
    "dropout_rate": ContinuousParameter(0.00, 0.2),
    "embedding_dimension": IntegerParameter(5, 50),
}

# objective_metric_name="test:mean_wQuantileLoss"
objective_metric_name = "test:mean_wQuantileLoss"

# Maximum number of models we will evaluate
max_jobs = 80
max_parallel_jobs = 5

tuner = HyperparameterTuner(
    estimator=estimator,
    objective_metric_name=objective_metric_name,
    hyperparameter_ranges=hyperparameter_ranges,
    max_jobs=max_jobs,
    strategy="Bayesian",
    random_seed=seed,
    objective_type="Minimize",
    max_parallel_jobs=max_parallel_jobs,
    early_stopping_type="Auto",
)

# Start hyperparameter tuning job
current_time = datetime.datetime.now().strftime("%d%m%Y%H%M")
tuning_job_name = f"{stack_parameters['project-prefix']}-{current_time}"
tuner.fit(inputs=data_channels, job_name=tuning_job_name)
tuner.wait()

### Get the Best Model

In [ ]:
best_training_job_name = tuner.best_training_job()
print(f" The name of the best training job is: {best_training_job_name}")
best_estimator = tuner.best_estimator()
print(
    f" The best estimator hyperparamters are: {json.dumps(best_estimator.hyperparameters(), indent=2)}"
)

## Create Endpoint and Predictor
A utility class is created to query the endpoint and perform predictions. This class is mainly copied from the AWS [DeepAR demo notebook](https://github.com/aws/amazon-sagemaker-examples/blob/main/introduction_to_amazon_algorithms/deepar_electricity/DeepAR-Electricity.ipynb)

__Note: Remember to delete the endpoint after running this experiment. A cell at the very bottom of this notebook will do that: make sure you run it at the end.__

Deploy the best estimator

In [ ]:
endpoint = tuner.deploy(
    initial_instance_count=1,
    endpoint_name=best_training_job_name,
    instance_type="ml.m5.xlarge",
    serializer=IdentitySerializer(content_type="application/json"),
    deserializer=JSONDeserializer(),
    wait=True,
)
predictor = DeepARPredictor(
    endpoint_name=endpoint.endpoint_name, sagemaker_session=sagemaker_session
)

## Make Predictions and Plot Results
We convert the train-test data to a Pandas Series and we use it to predict data points as defined in `prediction_length`.
The predicted data are compared to the validation dataset to evaluate the model.

In [ ]:
# Convert the context data to time series
ts = df.iloc[:-test_length, df.columns.get_loc(target_col)]
# Convert the validation dataframe to a time series
target_ts = df.iloc[-test_length:, df.columns.get_loc(target_col)]

df_predictions = predictor.predict(ts=ts, quantiles=[0.10, 0.5, 0.90])

In [ ]:
def plot(
    predictor,
    ts,
    target_ts,
    freq=pd.Timedelta(1, "min"),
    cat=None,
    show_samples=False,
    confidence=80,
):
    forecast_start_date = target_ts.index[0]
    print(
        "calling served model to generate predictions starting from {}".format(
            str(forecast_start_date)
        )
    )
    assert confidence > 50 and confidence < 100
    low_quantile = 0.5 - confidence * 0.005
    up_quantile = confidence * 0.005 + 0.5

    # we first construct the argument to call our model
    args = {
        "ts": ts,
        "return_samples": show_samples,
        "quantiles": [low_quantile, 0.5, up_quantile],
        "num_samples": 100,
    }

    plt.figure(figsize=(20, 3))
    ax = plt.subplot(1, 1, 1)

    if cat is not None:
        args["cat"] = cat
        ax.text(0.9, 0.9, "cat = {}".format(cat), transform=ax.transAxes)

    # call the end point to get the prediction
    prediction = predictor.predict(**args)

    # plot the samples
    if show_samples:
        for key in prediction.keys():
            if "sample" in key:
                prediction[key].plot(
                    color="lightskyblue", alpha=0.2, label="_nolegend_"
                )

    # plot the target
    target_ts.plot(color="black", label="target")

    # plot the confidence interval and the median predicted
    ax.fill_between(
        prediction[str(low_quantile)].index,
        prediction[str(low_quantile)].values,
        prediction[str(up_quantile)].values,
        color="b",
        alpha=0.3,
        label="{}% confidence interval".format(confidence),
    )
    prediction["0.5"].plot(color="b", label="P50")
    ax.legend(loc=2)

    # fix the scale as the samples may change it
    ax.set_ylim(target_ts.min() * 0.5, target_ts.max() * 1.5)

In [ ]:
plot(
    predictor,
    ts=ts,
    target_ts=target_ts,
    show_samples=False,
    confidence=60,
)

## Cleanup
Delete the model & endpoint used for testing.

In [ ]:
predictor.delete_model()
predictor.delete_endpoint()